In [ ]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
replace Schizophrenia/norm/S10W1.eea? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os

# Define the paths to your data folders using the correct folder name
schizophrenia_dir = '/content/Schizophrenia/sch/'
normal_dir = '/content/Schizophrenia/norm/'

# Get a list of all file names in each folder
schizophrenia_files = os.listdir(schizophrenia_dir)
normal_files = os.listdir(normal_dir)

print(f'Found {len(schizophrenia_files)} schizophrenia files.')
print(f'Found {len(normal_files)} normal files.')

Found 45 schizophrenia files.
Found 39 normal files.


In [ ]:
import pandas as pd

# Load one file to begin the cleaning process
file_path = '/content/Schizophrenia/norm/S10W1.eea'
df = pd.read_csv(file_path, header=None)

print(df.head())

        0
0  347.78
1  507.87
2  488.54
3  369.86
4  347.78


agumenting the data to avoid overfitting

In [ ]:
import numpy as np

def augment_eeg_data(data, noise_level=0.01):
    """
    Augments the EEG data by adding Gaussian noise.
    :param data: A NumPy array of your EEG data (e.g., all_data list from your code)
    :param noise_level: The standard deviation of the noise to add
    :return: An augmented list of EEG data arrays
    """
    augmented_data = []

    for subject_data in data:
        # Create Gaussian noise with the same shape as the subject's data
        noise = np.random.normal(0, noise_level, subject_data.shape)

        # Add the noise to the original data
        noisy_data = subject_data + noise

        # Add both the original and augmented data to the new list
        augmented_data.append(subject_data)
        augmented_data.append(noisy_data)

    return augmented_data

for single file the feature is extrated

In [ ]:
import pandas as pd
import numpy as np

# Load the file
file_path = '/content/Schizophrenia/norm/S10W1.eea'
df = pd.read_csv(file_path, header=None)

# Convert to a NumPy array
data_array = df.values

# Define the number of channels
n_channels = 16

# Reshape the data using -1 to let NumPy figure out the number of samples
reshaped_data = data_array.reshape(-1, n_channels)

# Print the new shape to see the corrected number of samples
print("Original shape:", data_array.shape)
print("Corrected reshaped shape:", reshaped_data.shape)

Original shape: (122880, 1)
Corrected reshaped shape: (7680, 16)


for all the file the featured is extrated

In [ ]:
import os
import pandas as pd
import numpy as np

# Define the paths to your data folders
schizophrenia_dir = '/content/Schizophrenia/sch/'
normal_dir = '/content/Schizophrenia/norm/'

# Define the number of channels
n_channels = 16

# Create empty lists to store all the reshaped data and labels
all_data = []
all_labels = []

# --- Loop through all schizophrenia files ---
for filename in os.listdir(schizophrenia_dir):
    if filename.endswith('.eea'):
        file_path = os.path.join(schizophrenia_dir, filename)

        # Load the data
        df = pd.read_csv(file_path, header=None)

        # Reshape the data
        data_array = df.values
        reshaped_data = data_array.reshape(-1, n_channels)

        # Add the reshaped data and its label to the lists
        all_data.append(reshaped_data)
        all_labels.append(1) # Label for schizophrenia (1)

# --- Loop through all normal files ---
for filename in os.listdir(normal_dir):
    if filename.endswith('.eea'):
        file_path = os.path.join(normal_dir, filename)

        # Load the data
        df = pd.read_csv(file_path, header=None)

        # Reshape the data
        data_array = df.values
        reshaped_data = data_array.reshape(-1, n_channels)

        # Add the reshaped data and its label to the lists
        all_data.append(reshaped_data)
        all_labels.append(0) # Label for normal (0)

print(f"Total files processed: {len(all_data)}")
print(f"Total labels created: {len(all_labels)}")

Total files processed: 84
Total labels created: 84


now extreding the features

In [ ]:
import numpy as np
from scipy.fftpack import rfft, fftfreq
import pywt

# Your feature extraction function
def extract_features(eeg_data, n_channels=16, sample_rate=128):
    all_features = []
    # Loop through each subject's data in the list
    for subject_data in eeg_data:
        subject_features = []
        # Loop through each of the 16 channels
        for channel_data in subject_data.T:

            # --- Fourier Transform Features (Frequency Power) ---
            N = len(channel_data)
            yf = rfft(channel_data)
            xf = fftfreq(N, 1 / sample_rate)
            alpha_power = np.sum(np.abs(yf[(xf > 8) & (xf < 12)]))
            subject_features.append(alpha_power)

            # --- Wavelet Transform Features (Decomposition) ---
            coeffs = pywt.wavedec(channel_data, 'db4', level=5)
            subject_features.extend(coeffs[0])

        all_features.append(np.array(subject_features))

    return np.array(all_features)

# Call the function to process all your data
X = extract_features(all_data)
y = np.array(all_labels)

print(f"Final data shape (X): {X.shape}")
print(f"Final labels shape (y): {y.shape}")

Final data shape (X): (84, 3952)
Final labels shape (y): (84,)


now training the model final step

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Apply MinMaxScaler to scale your data to a small range (0-1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Reshape data for the CNN (samples, features, 1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid') # Use 'sigmoid' for binary classification
])

# Compile and train the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# The model will train for 20 epochs (cycles through the data)
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 222ms/step - accuracy: 0.5005 - loss: 1.6275 - val_accuracy: 0.5294 - val_loss: 1.2582
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5080 - loss: 1.1024 - val_accuracy: 0.4706 - val_loss: 0.8159
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.5339 - loss: 0.6958 - val_accuracy: 0.5294 - val_loss: 1.1665
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.5226 - loss: 1.2003 - val_accuracy: 0.5294 - val_loss: 0.9133
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5375 - loss: 0.8057 - val_accuracy: 0.4706 - val_loss: 0.8445
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.4419 - loss: 0.8120 - val_accuracy: 0.4706 - val_loss: 0.7469
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 0.6405 - loss: 0.6125 - val_accuracy: 0.5294 - val_loss: 0.7159
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.6377 - loss: 0.5463 - val_accuracy: 0.5294 - val_loss: 0.7020
Epo

the model is overfitted now using the techique of early stopping here

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),

    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.5), # Add dropout to prevent overfitting

    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with the new architecture
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 607ms/step - accuracy: 0.4810 - loss: 3.4320 - val_accuracy: 0.4706 - val_loss: 0.8612
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 405ms/step - accuracy: 0.6526 - loss: 3.2071 - val_accuracy: 0.5294 - val_loss: 0.7163
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 336ms/step - accuracy: 0.6639 - loss: 7.1235 - val_accuracy: 0.5294 - val_loss: 0.6889
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 278ms/step - accuracy: 0.8281 - loss: 1.5862 - val_accuracy: 0.4706 - val_loss: 0.6955
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 300ms/step - accuracy: 0.7901 - loss: 1.9569 - val_accuracy: 0.5882 - val_loss: 0.7106
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 282ms/step - accuracy: 0.8203 - loss: 1.6818 - val_accuracy: 0.4706 - val_loss: 0.7745
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 293ms/step - accuracy: 0.9005 - loss: 0.6223 - val_accuracy: 0.4706 - val_loss: 0.9326
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 298ms/step - accuracy: 0.8973 - loss: 0.8799 - val_accuracy: 0.4706 - val_loss:

again the model is overfitted now using simpler model for this

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 175ms/step - accuracy: 0.5265 - loss: 0.7613 - val_accuracy: 0.5294 - val_loss: 0.6831
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4885 - loss: 0.6880 - val_accuracy: 0.4706 - val_loss: 0.6932
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4696 - loss: 0.6932 - val_accuracy: 0.4706 - val_loss: 0.6932
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4852 - loss: 0.6932 - val_accuracy: 0.4706 - val_loss: 0.6932
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4579 - loss: 0.6932 - val_accuracy: 0.4706 - val_loss: 0.6932
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.4735 - loss: 0.6932 - val_accuracy: 0.4706 - val_loss: 0.6932
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4892 - loss: 0.6931 - val_accuracy: 0.5294 - val_loss: 0.6931
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5538 - loss: 0.6931 - val_accuracy: 0.5294 - val_loss: 0.6931

end


re doing with the model

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.fftpack import rfft, fftfreq
from scipy.stats import skew, kurtosis
import pywt

# Your data loading and reshaping from before
# Make sure to run the code that creates `all_data` and `all_labels` first.
# Example:
# all_data = [...]
# all_labels = [...]

# --- UPDATED Feature Extraction Function ---
def extract_robust_features(eeg_data, n_channels=16, sample_rate=128):
    all_features = []
    # Loop through each subject's data in the list
    for subject_data in eeg_data:
        subject_features = []
        # Loop through each of the 16 channels
        for channel_data in subject_data.T:

            # --- Time-Domain Statistical Features ---
            subject_features.append(np.mean(channel_data))
            subject_features.append(np.std(channel_data))
            subject_features.append(skew(channel_data))
            subject_features.append(kurtosis(channel_data))

            # --- Frequency-Domain Features (Power in different bands) ---
            N = len(channel_data)
            yf = rfft(channel_data)
            xf = fftfreq(N, 1 / sample_rate)

            # Define frequency bands
            bands = {
                'delta': (1, 4),
                'theta': (4, 8),
                'alpha': (8, 12),
                'beta': (12, 30),
                'gamma': (30, 45)
            }

            for band_name, (low, high) in bands.items():
                band_power = np.sum(np.abs(yf[(xf > low) & (xf < high)]))
                subject_features.append(band_power)

            # --- Wavelet Transform Features (Decomposition) ---
            # Using the same approach as before
            coeffs = pywt.wavedec(channel_data, 'db4', level=5)
            subject_features.extend(coeffs[0])

        all_features.append(np.array(subject_features))

    return np.array(all_features)

# Call the new function to process all your data
X = extract_robust_features(all_data)
y = np.array(all_labels)

print(f"Final data shape (X): {X.shape}")
print(f"Final labels shape (y): {y.shape}")

Final data shape (X): (84, 4080)
Final labels shape (y): (84,)


trainignt he deep learning model again

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Apply MinMaxScaler to scale your data to a small range (0-1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Reshape data for the CNN (samples, features, 1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build a more robust CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),

    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile and train the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# The model will train for 30 epochs
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 435ms/step - accuracy: 0.5304 - loss: 11.2725 - val_accuracy: 0.5294 - val_loss: 0.6526
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 301ms/step - accuracy: 0.6952 - loss: 3.2711 - val_accuracy: 0.4706 - val_loss: 0.7035
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 284ms/step - accuracy: 0.5105 - loss: 3.4199 - val_accuracy: 0.4706 - val_loss: 0.6986
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 380ms/step - accuracy: 0.6718 - loss: 2.1636 - val_accuracy: 0.5294 - val_loss: 0.7050
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 453ms/step - accuracy: 0.6302 - loss: 2.1849 - val_accuracy: 0.5294 - val_loss: 0.8836
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 295ms/step - accuracy: 0.6981 - loss: 0.9100 - val_accuracy: 0.4706 - val_loss: 1.1922
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - accuracy: 0.7709 - loss: 0.6327 - val_accuracy: 0.4706 - val_loss: 1.5894
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 305ms/step - accuracy: 0.8203 - loss: 0.4611 - val_accuracy: 0.4706 - val_loss: 1.8958
Ep

In [ ]:
from scipy.fftpack import rfft, fftfreq
import pywt
import numpy as np

def extract_features(eeg_data, n_channels=16, sample_rate=128):
    all_features = []
    # Loop through each subject's data
    for subject_data in eeg_data:
        subject_features = []
        # Loop through each of the 16 channels
        for channel_data in subject_data.T:

            # --- Fourier Transform Features (Frequency Power) ---
            N = len(channel_data)
            yf = rfft(channel_data)
            xf = fftfreq(N, 1 / sample_rate)
            # Example: Get the power of the Alpha band (8-12 Hz)
            alpha_power = np.sum(np.abs(yf[(xf > 8) & (xf < 12)]))
            subject_features.append(alpha_power)

            # --- Wavelet Transform Features (Decomposition) ---
            # Decompose the signal using a wavelet
            coeffs = pywt.wavedec(channel_data, 'db4', level=5)
            # Use the approximation coefficients as a feature
            subject_features.extend(coeffs[0])

        all_features.append(np.array(subject_features))

    return np.array(all_features)

# This is the reshaped data from your previous step
# You will need to apply the cleaning and reshaping to all your files first
# before running this step.
# For now, you can test it on your single reshaped_data array
X_features = extract_features(reshaped_data[np.newaxis, :])

print(f"Shape of extracted features: {X_features.shape}")

Shape of extracted features: (1, 3952)


In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.fftpack import rfft, fftfreq
import pywt

# Define the paths to your data folders
schizophrenia_dir = '/content/Schizophrenia/sch/'
normal_dir = '/content/Schizophrenia/norm/'

# --- Your feature extraction function from before ---
def extract_features(eeg_data, n_channels=16, sample_rate=128):
    all_features = []
    for subject_data in eeg_data:
        subject_features = []
        for channel_data in subject_data.T:
            N = len(channel_data)
            yf = rfft(channel_data)
            xf = fftfreq(N, 1 / sample_rate)
            alpha_power = np.sum(np.abs(yf[(xf > 8) & (xf < 12)]))
            subject_features.append(alpha_power)
            coeffs = pywt.wavedec(channel_data, 'db4', level=5)
            subject_features.extend(coeffs[0])
        all_features.append(np.array(subject_features))
    return np.array(all_features)
# ---------------------------------------------------

# Create lists to store data and labels for all subjects
all_subject_features = []
all_subject_labels = []
n_channels = 16

# Process Schizophrenia files
for filename in os.listdir(schizophrenia_dir):
    if filename.endswith('.eea'):
        file_path = os.path.join(schizophrenia_dir, filename)
        df = pd.read_csv(file_path, header=None)
        # Reshape data with an auto-calculated number of samples
        reshaped_data = df.values.reshape(-1, n_channels)
        # Extract features and append to lists
        features = extract_features(reshaped_data[np.newaxis, :])
        all_subject_features.append(features[0])
        all_subject_labels.append(1)

# Process Normal files
for filename in os.listdir(normal_dir):
    if filename.endswith('.eea'):
        file_path = os.path.join(normal_dir, filename)
        df = pd.read_csv(file_path, header=None)
        reshaped_data = df.values.reshape(-1, n_channels)
        features = extract_features(reshaped_data[np.newaxis, :])
        all_subject_features.append(features[0])
        all_subject_labels.append(0)

# Convert lists to NumPy arrays for the model
X = np.array(all_subject_features)
y = np.array(all_subject_labels)

print(f"Final data shape (X): {X.shape}")
print(f"Final labels shape (y): {y.shape}")

Final data shape (X): (84, 3952)
Final labels shape (y): (84,)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Reshape data for the CNN (samples, features, 1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile and train the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step - accuracy: 0.4735 - loss: 299143.3125 - val_accuracy: 0.4706 - val_loss: 404914.7188
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.4966 - loss: 225117.8438 - val_accuracy: 0.5294 - val_loss: 551.1238
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.5304 - loss: 636.8221 - val_accuracy: 0.5294 - val_loss: 0.6931
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.5304 - loss: 0.6931 - val_accuracy: 0.5294 - val_loss: 0.6931
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5304 - loss: 0.6931 - val_accuracy: 0.5294 - val_loss: 0.6931
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.5538 - loss: 0.6930 - val_accuracy: 0.5294 - val_loss: 0.6930
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.5187 - loss: 0.6931 - val_accuracy: 0.5294 - val_loss: 0.6930
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.5343 - loss: 0.6930 - val_accuracy:

In [ ]:
# The shapes from your previous step are used here
num_samples = X_scaled.shape[0]
num_features = X_scaled.shape[1]

# Reshape the entire dataset first
X_reshaped_for_cnn = X_scaled.reshape(num_samples, num_features, 1)

# Now, split the reshaped data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped_for_cnn, y, test_size=0.2, random_state=42
)

# Check the final shapes before feeding to the model
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

ValueError: Found input variables with inconsistent numbers of samples: [1, 84]

**3rd try**

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.fftpack import rfft, fftfreq
from scipy.stats import skew, kurtosis
import pywt

# Define the paths to your data folders
schizophrenia_dir = '/content/Schizophrenia/sch/'
normal_dir = '/content/Schizophrenia/norm/'

# Define the number of channels and sample rate
n_channels = 16
sample_rate = 128

# Create empty lists to store all the data and labels
all_data = []
all_labels = []

# --- Loop through schizophrenia files ---
for filename in os.listdir(schizophrenia_dir):
    if filename.endswith('.eea'):
        file_path = os.path.join(schizophrenia_dir, filename)
        df = pd.read_csv(file_path, header=None)
        reshaped_data = df.values.reshape(-1, n_channels)
        all_data.append(reshaped_data)
        all_labels.append(1)  # Label for schizophrenia

# --- Loop through normal files ---
for filename in os.listdir(normal_dir):
    if filename.endswith('.eea'):
        file_path = os.path.join(normal_dir, filename)
        df = pd.read_csv(file_path, header=None)
        reshaped_data = df.values.reshape(-1, n_channels)
        all_data.append(reshaped_data)
        all_labels.append(0)  # Label for normal

def extract_robust_features(eeg_data):
    all_features = []
    for subject_data in eeg_data:
        subject_features = []
        for channel_data in subject_data.T:
            # Time-Domain Statistical Features
            subject_features.append(np.mean(channel_data))
            subject_features.append(np.std(channel_data))
            subject_features.append(skew(channel_data))
            subject_features.append(kurtosis(channel_data))

            # Frequency-Domain Features
            N = len(channel_data)
            yf = rfft(channel_data)
            xf = fftfreq(N, 1 / sample_rate)
            bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 45)}
            for low, high in bands.values():
                band_power = np.sum(np.abs(yf[(xf > low) & (xf < high)]))
                subject_features.append(band_power)

            # Wavelet Features
            coeffs = pywt.wavedec(channel_data, 'db4', level=5)
            subject_features.extend(coeffs[0])

        all_features.append(np.array(subject_features))

    return np.array(all_features)

# Extract features from all processed data
X = extract_robust_features(all_data)
y = np.array(all_labels)

print(f"Final data shape (X): {X.shape}")
print(f"Final labels shape (y): {y.shape}")

Final data shape (X): (84, 4080)
Final labels shape (y): (84,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
# test_size=0.2 means 20% of the data will be used for testing
# random_state ensures the split is the same every time you run the code
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print("\n--- After Splitting ---")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Optional: Check the label distribution
print("\n--- Label Distribution ---")
print("Training labels:", np.bincount(y_train))
print("Testing labels:", np.bincount(y_test))


--- After Splitting ---
X_train shape: (67, 4080)
y_train shape: (67,)
X_test shape: (17, 4080)
y_test shape: (17,)

--- Label Distribution ---
Training labels: [31 36]
Testing labels: [8 9]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Random Forest model
# We'll use 100 trees in the forest
# random_state ensures you get the same results every time
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on your training data
# This is where the model learns to find patterns
model.fit(X_train, y_train)

# Make predictions on the test data
# The model tries to classify the data it has never seen before
y_pred = model.predict(X_test)

# Evaluate the model's performance
# We'll use accuracy and a full classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.4117647058823529

Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.25      0.29         8
           1       0.45      0.56      0.50         9

    accuracy                           0.41        17
   macro avg       0.39      0.40      0.39        17
weighted avg       0.40      0.41      0.40        17

